# Классификация обзоров фильмов



# Цель
Классификация последовательностей - это проблема прогнозирующего моделирования, когда у вас есть некоторая последовательность входных данных в пространстве или времени, и задача состоит в том, чтобы предсказать категорию для последовательности.
Проблема усложняется тем, что последовательности могут различаться по длине, состоять из очень большого словарного запаса входных символов и могут потребовать от модели изучения долгосрочного контекста или зависимостей между символами во входной последовательности. 

В данной лабораторной работе также будет использоваться датасет IMDb, однако
обучение будет проводиться с помощью рекуррентной нейронной сети.

# Задачи

* Ознакомиться с рекуррентными нейронными сетями
* Изучить способы классификации текста
* Ознакомиться с ансамблированием сетей
* Построить ансамбль сетей, который позволит получать точность не менее 97%

# Выполнение работы


### Подключение модулей
Набор данных imdb уже входит в состав Keras в форме набора из четырех массивов Numpy.

In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot
from keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

#Для стакинга моделей 
from tensorflow.keras.models import load_model
from numpy import dstack
from os import makedirs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

Напишем функцию для загрузки тренеровочных и тестовых данных

In [2]:
def load_dataset():
    # Загружаем данные
    (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

    print('x_train shape:',x_train.shape)
    print('x_test shape:',x_test.shape)

    return x_train, y_train, x_test, y_test

Создадим функцию которая преобразует наши данные в необходимы вид

In [3]:
def prep_pixels(train, test):
    train_norm = pad_sequences(train, maxlen=100, value = 0.0)
    test_norm = pad_sequences(test, maxlen=100, value = 0.0)
    return train_norm, test_norm

Создадим модель обучения.

In [4]:
def define_model():
    model = Sequential()
    #Превращает положительные целые числа (индексы) в плотные векторы фиксированного размера.
    model.add(Embedding(10000, 50, input_length=100))
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))
    
    model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
    model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))
    
    model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
    model.add(Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.4))

    #model.add(Flatten())
    model.add(Dense(128, activation = "relu"))
    model.add(Dropout(0.3))
    
    model.add(LSTM(128))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    #model.summary()
    return model

Функция прогона и сохранения моделей

In [5]:
def save_all_modelss():

    trainX, trainY, testX, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    
    # create directory for models
    makedirs('models')
    # fit and save models
    n_members = 2
    for i in range(n_members):
        # fit model
        model = define_model()   
        #Обучение сети
        model.fit(trainX, trainY, epochs=3, batch_size=508, validation_data=(testX, testY), verbose=1)
 
        # save model
        filename = 'models/model_' + str(i + 1) + '.h5'
        model.save(filename)
        print('>Saved %s' % filename)

Натренируем модели

In [6]:
save_all_modelss()

x_train shape: (25000,)
x_test shape: (25000,)
Epoch 1/3
50/50 [==============================] - 17s 89ms/step - loss: 0.6794 - accuracy: 0.5378 - val_loss: 0.5585 - val_accuracy: 0.7037
Epoch 2/3
50/50 [==============================] - 3s 53ms/step - loss: 0.4466 - accuracy: 0.7922 - val_loss: 0.4516 - val_accuracy: 0.7824
Epoch 3/3
50/50 [==============================] - 3s 53ms/step - loss: 0.3102 - accuracy: 0.8671 - val_loss: 0.3825 - val_accuracy: 0.8288
>Saved models/model_1.h5
Epoch 1/3
50/50 [==============================] - 7s 70ms/step - loss: 0.6861 - accuracy: 0.5322 - val_loss: 0.5555 - val_accuracy: 0.7314
Epoch 2/3
50/50 [==============================] - 3s 53ms/step - loss: 0.4621 - accuracy: 0.7806 - val_loss: 0.4976 - val_accuracy: 0.7625
Epoch 3/3
50/50 [==============================] - 3s 53ms/step - loss: 0.3270 - accuracy: 0.8587 - val_loss: 0.4852 - val_accuracy: 0.7846
>Saved models/model_2.h5


Создадим функцию загрузки моделей

In [7]:
def load_all_models(n_models):
    all_models = list()
    for i in range(n_models):
        # define filename for this ensemble
        filename = 'models/model_' + str(i + 1) + '.h5'
        # load model from file
        model = load_model(filename)
        # add to list of members
        all_models.append(model)
        print('>loaded %s' % filename)
    return all_models

Создадим функцию зборки моделей

In [8]:
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        print ("Predict",yhat)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat
        else:
            stackX = dstack((stackX, yhat))
            print ("Stack predict", stackX)
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

Функция обучения со всеми моделями

In [9]:
def fit_stacked_model(members, inputX, inputY):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit standalone model
    model = LogisticRegression()
    model.fit(stackedX, inputY)
    return model

Функция предсказания

In [10]:
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

Функцию, которая позволяет ввести пользовательский текст

In [11]:
def textToInt(text, dimension = 10000):
    indexes = imdb.get_word_index()
    temp = []
    for word in text:
        try:
            if (indexes[word] < 9998):
                temp.append(indexes[word] + 3)
        except Exception:
            print("Введен недопустимый символ")
            temp.append(0)       
            continue
    while len(temp) < 100:
        temp.insert(0,0)
    temp = np.array(temp).reshape(-1, 100)
    return temp

Функция запуска нашей нейроной сети

In [12]:
def run_test_harness():
    trainX, trainY, testX, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    
    #Загрузка моделей
    n_members = 2
    members = load_all_models(n_members)
    
    print('Loaded %d models' % len(members))
    
    for model in members:
        loss, acc = model.evaluate(testX, testY, verbose=0)
        print('Model Accuracy: %.3f' % acc)
    
    model = fit_stacked_model(members, testX, testY)
    
    return members, model


Ансамблируем модель

In [13]:
members, model = run_test_harness()

x_train shape: (25000,)
x_test shape: (25000,)
>loaded models/model_1.h5
>loaded models/model_2.h5
Loaded 2 models
Model Accuracy: 0.829
Model Accuracy: 0.785
Predict [[0.0549113 ]
 [0.952798  ]
 [0.76942307]
 ...
 [0.01294683]
 [0.02541268]
 [0.19453388]]
Predict [[0.86047083]
 [0.98091024]
 [0.96378964]
 ...
 [0.07352982]
 [0.51022553]
 [0.9286893 ]]
Stack predict [[[0.0549113  0.86047083]]

 [[0.952798   0.98091024]]

 [[0.76942307 0.96378964]]

 ...

 [[0.01294683 0.07352982]]

 [[0.02541268 0.51022553]]

 [[0.19453388 0.9286893 ]]]


Ввод и предикт текста

In [14]:
def inp_in_ense(members, model):
    #Ввод пользователем текст.
    indexesList = textToInt(input().split(" "))
    print ("-----------")
    print (indexesList)
    print ("-----------")
    il = pad_sequences(indexesList, maxlen=100, value = 0.1)
    print (il)
    print ("-----------")
    #t = np.transpose(indexesList)
    #a = model.predict(t) 
    a = stacked_prediction(members, model, il)
    #acc = accuracy_score(testY,a)
    print('Stacked Test Accuracy: %.3f' % a)

In [17]:
inp_in_ense(members, model)

very good movie i am fine
-----------
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  55  52  20  13 244 478]]
-----------
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  55  52  20  13 244 478]]
-----------
Predict [[0.30966976]]
Predict [[0.98690665]]
Stack predict [[[0.30966976 0.98690665]]]
Stacked Test Accuracy:

In [18]:
inp_in_ense(members, model)

very bad movie
-----------
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0 55 78 20]]
-----------
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0 55 78 20]]
-----------
Predict [[0.04598336]]
Predict [[0.9501602]]
Stack predict [[[0.04598336 0.9501602 ]]]
Stacked Test Accuracy: 0.000
